In [1]:
!pip install --quiet google-cloud-aiplatform google-cloud-storage vertexai pandas numpy langchain tiktoken python-dateutil sentence-transformers scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 11.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.13.0 requires google-cloud-aiplatform[agent-engines]<2.0.0,>=1.95.1, but you have google-cloud-aiplatform 1.71.1 which is incompatible.


In [4]:
!gcloud auth login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=1wqJ1m9qr6bhzKMGFyvGRmqmDJ4YBp&prompt=consent&token_usage=remote&access_type=offline&code_challenge=5cZGdOWUEjYeb2Zi6j9ADV5--Z_LLCmJDh3O4jRSeYU&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AVMBsJgJKMSW5cAhnEAfmTcSKz0PCmFTgIMnxpq2xDQLUAl3evO3oQABc7QLcCzlPuKlVw

You are now logged in as [jonahprashanth@gmail.com].
Your current proje

In [6]:
!gcloud config set project i-monolith-468706-i9

Updated property [core/project].


In [7]:
!gcloud auth list

     Credentialed Accounts
ACTIVE  ACCOUNT
*       jonahprashanth@gmail.com

To set the active account, run:
    $ gcloud config set account `ACCOUNT`



In [8]:
!gcloud config list

[component_manager]
disable_update_check = True
[compute]
gce_metadata_read_timeout_sec = 0
[core]
account = jonahprashanth@gmail.com
project = i-monolith-468706-i9

Your active configuration is: [default]


In [10]:
!gcloud iam service-accounts create vertex-sa \
  --display-name "Vertex AI Service Account" \
  --project i-monolith-468706-i9

Created service account [vertex-sa].


In [11]:
!gcloud projects add-iam-policy-binding i-monolith-468706-i9 \
  --member="serviceAccount:vertex-sa@i-monolith-468706-i9.iam.gserviceaccount.com" \
  --role="roles/aiplatform.user"

!gcloud projects add-iam-policy-binding i-monolith-468706-i9 \
  --member="serviceAccount:vertex-sa@i-monolith-468706-i9.iam.gserviceaccount.com" \
  --role="roles/storage.objectAdmin"


Updated IAM policy for project [i-monolith-468706-i9].
bindings:
- members:
  - serviceAccount:sample-test@i-monolith-468706-i9.iam.gserviceaccount.com
  role: roles/aiplatform.admin
- members:
  - serviceAccount:service-393707886745@gcp-sa-aiplatform.iam.gserviceaccount.com
  role: roles/aiplatform.serviceAgent
- members:
  - serviceAccount:sample-test@i-monolith-468706-i9.iam.gserviceaccount.com
  - serviceAccount:vertex-sa@i-monolith-468706-i9.iam.gserviceaccount.com
  - user:jonahprashanth@gmail.com
  role: roles/aiplatform.user
- members:
  - serviceAccount:service-393707886745@gcp-sa-artifactregistry.iam.gserviceaccount.com
  role: roles/artifactregistry.serviceAgent
- members:
  - serviceAccount:393707886745@cloudbuild.gserviceaccount.com
  role: roles/cloudbuild.builds.builder
- members:
  - serviceAccount:service-393707886745@gcp-sa-cloudbuild.iam.gserviceaccount.com
  role: roles/cloudbuild.serviceAgent
- members:
  - serviceAccount:service-393707886745@compute-system.iam.gse

In [14]:
!gcloud services enable aiplatform.googleapis.com
!gcloud services enable storage.googleapis.com

In [12]:
!gcloud iam service-accounts keys create key.json \
  --iam-account=vertex-sa@i-monolith-468706-i9.iam.gserviceaccount.com


created key [8be270412f1ca8482bb84050b786bfa5ab63f7b5] of type [json] as [key.json] for [vertex-sa@i-monolith-468706-i9.iam.gserviceaccount.com]


In [13]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "key.json"


In [19]:
import os
import json
import pandas as pd
import numpy as np
import re
from datetime import datetime
from typing import List, Dict, Any, Optional
import ast
import warnings
import time
from sklearn.metrics.pairwise import cosine_similarity
from google.cloud import storage
import vertexai
from vertexai.preview.language_models import TextEmbeddingModel
from vertexai.generative_models import GenerativeModel
from langchain.text_splitter import RecursiveCharacterTextSplitter

warnings.filterwarnings('ignore')

# Configuration
PROJECT_ID = "i-monolith-468706-i9"
LOCATION = "us-central1"
BUCKET_NAME = "bucket_jonah"  # Update with your bucket name

# Initialize Vertex AI
vertexai.init(project=PROJECT_ID, location=LOCATION)

class MedicalDataProcessor:
    """Handles CSV data cleaning and preprocessing"""

    def __init__(self):
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1500,  # 1.5KB chunks
            chunk_overlap=200,
            separators=["\n\n", "\n", ". ", " ", ""]
        )

    def clean_csv_data(self, csv_path: str) -> pd.DataFrame:
        """Clean and preprocess the EHR CSV data"""
        print("Loading CSV data...")
        df = pd.read_csv(csv_path)

        print(f"Original data shape: {df.shape}")

        # Basic cleaning
        df = df.dropna(subset=['TEXT'])  # Remove rows without medical text
        df = df.drop_duplicates(subset=['TEXT'])  # Remove duplicate notes

        # Clean text content
        df['TEXT'] = df['TEXT'].apply(self._clean_medical_text)

        # Parse JSON fields safely
        df['diagnoses_parsed'] = df['diagnoses'].apply(self._safe_parse_json)
        df['procedures_parsed'] = df['procedures'].apply(self._safe_parse_json)
        df['cpt_codes_parsed'] = df['cpt_codes'].apply(self._safe_parse_json)

        # Standardize dates
        df['CHARTDATE'] = pd.to_datetime(df['CHARTDATE'], errors='coerce')

        # Remove rows with no medical codes
        df = df[
            (df['diagnoses_parsed'].notna()) |
            (df['procedures_parsed'].notna()) |
            (df['cpt_codes_parsed'].notna())
        ]

        print(f"Cleaned data shape: {df.shape}")
        return df

    def _clean_medical_text(self, text: str) -> str:
        """Clean medical text content"""
        if pd.isna(text) or not isinstance(text, str):
            return ""

        # Remove common PHI patterns (basic anonymization)
        text = re.sub(r'\[\*\*.*?\*\*\]', '[REDACTED]', text)  # Remove bracketed PHI

        # Normalize whitespace
        text = re.sub(r'\s+', ' ', text).strip()

        # Remove control characters
        text = ''.join(char for char in text if ord(char) >= 32 or char == '\n')

        return text

    def _safe_parse_json(self, json_str: str) -> Optional[List[Dict]]:
        """Safely parse JSON strings from CSV"""
        if pd.isna(json_str) or json_str == '':
            return None

        try:
            # Handle string representation of list
            if isinstance(json_str, str):
                # Try ast.literal_eval first
                parsed = ast.literal_eval(json_str)
                return parsed if isinstance(parsed, list) else None
        except (ValueError, SyntaxError):
            try:
                # Try json.loads as backup
                return json.loads(json_str)
            except json.JSONDecodeError:
                return None
        return None

    def _extract_codes_as_strings(self, codes_list: Optional[List[Dict]], code_key: str) -> List[str]:
        """Extract codes from parsed JSON and convert to strings"""
        if not codes_list:
            return []

        result = []
        for item in codes_list:
            if isinstance(item, dict) and code_key in item:
                code = item[code_key]
                if code is not None and str(code).strip():
                    result.append(str(code).strip())
        return result

    def _extract_titles_as_strings(self, codes_list: Optional[List[Dict]], title_key: str) -> List[str]:
        """Extract titles from parsed JSON and convert to strings"""
        if not codes_list:
            return []

        result = []
        for item in codes_list:
            if isinstance(item, dict) and title_key in item:
                title = item[title_key]
                if title is not None and str(title).strip():
                    result.append(str(title).strip())
        return result

    def chunk_documents(self, df: pd.DataFrame) -> List[Dict[str, Any]]:
        """Split long medical texts into chunks"""
        chunks = []

        for idx, row in df.iterrows():
            text = row['TEXT']
            if not text or len(text.strip()) < 50:
                continue

            # Split text into chunks
            text_chunks = self.text_splitter.split_text(text)

            for chunk_idx, chunk in enumerate(text_chunks):
                chunk_data = {
                    'content': chunk,
                    'original_row_id': int(row['row_id']),
                    'subject_id': int(row['SUBJECT_ID']),
                    'admission_id': int(row['HADM_ID']),
                    'chart_date': row['CHARTDATE'].isoformat() if pd.notna(row['CHARTDATE']) else None,
                    'category': str(row['CATEGORY']),
                    'description': str(row['DESCRIPTION']),
                    'chunk_id': chunk_idx,
                    'total_chunks': len(text_chunks),

                    # Medical codes
                    'diagnoses': row['diagnoses_parsed'] or [],
                    'procedures': row['procedures_parsed'] or [],
                    'cpt_codes': row['cpt_codes_parsed'] or [],

                    # Extract code strings properly
                    'diagnosis_codes': self._extract_codes_as_strings(row['diagnoses_parsed'], 'ICD9_CODE'),
                    'diagnosis_titles': self._extract_titles_as_strings(row['diagnoses_parsed'], 'LONG_TITLE'),
                    'procedure_codes': self._extract_codes_as_strings(row['procedures_parsed'], 'ICD9_CODE'),
                    'procedure_titles': self._extract_titles_as_strings(row['procedures_parsed'], 'LONG_TITLE'),
                    'cpt_code_values': self._extract_codes_as_strings(row['cpt_codes_parsed'], 'CPT_CD'),
                    'cpt_descriptions': self._extract_titles_as_strings(row['cpt_codes_parsed'], 'DESCRIPTION')
                }
                chunks.append(chunk_data)

        print(f"Created {len(chunks)} text chunks")
        return chunks

class VertexAIMedicalRAG:
    """Medical RAG system using Google Cloud Storage and Vertex AI"""

    def __init__(self, project_id: str, location: str, bucket_name: str):
        self.project_id = project_id
        self.location = location
        self.bucket_name = bucket_name

        # Initialize Vertex AI
        vertexai.init(project=project_id, location=location)

        # Initialize models
        self.embedding_model = TextEmbeddingModel.from_pretrained("text-embedding-004")
        self.generative_model = GenerativeModel("gemini-1.5-flash-002")

        # Initialize GCS client
        self.storage_client = storage.Client(project=project_id)
        self.bucket = self.storage_client.bucket(bucket_name)

        # In-memory storage for embeddings (loaded from GCS)
        self.medical_records = []

        print(f"✅ Initialized Vertex AI Medical RAG")
        print(f"   Project: {project_id}")
        print(f"   Location: {location}")
        print(f"   Bucket: {bucket_name}")

    def generate_embeddings(self, texts: List[str], batch_size: int = 32) -> List[List[float]]:
        """Generate embeddings using Vertex AI in batches"""
        all_embeddings = []

        print(f"Generating embeddings for {len(texts)} texts in batches of {batch_size}...")

        for i in range(0, len(texts), batch_size):
            batch = texts[i:i+batch_size]
            try:
                embeddings = self.embedding_model.get_embeddings(batch)
                batch_embeddings = [e.values for e in embeddings]
                all_embeddings.extend(batch_embeddings)

                print(f"  Processed batch {i//batch_size + 1}/{(len(texts)-1)//batch_size + 1}")
                time.sleep(0.5)  # Rate limiting

            except Exception as e:
                print(f"❌ Error in batch {i//batch_size + 1}: {e}")
                # Add empty embeddings for failed batch
                empty_embeddings = [[0.0] * 768 for _ in batch]  # text-embedding-004 has 768 dimensions
                all_embeddings.extend(empty_embeddings)

        return all_embeddings

    def create_embeddings_and_store(self, chunks: List[Dict[str, Any]]):
        """Create embeddings for chunks and store in GCS"""
        print("=== EMBEDDING CREATION PHASE ===")

        # Extract texts for embedding
        texts = [chunk['content'] for chunk in chunks]

        # Generate embeddings
        embeddings = self.generate_embeddings(texts)

        # Combine chunks with embeddings
        medical_records = []
        for chunk, embedding in zip(chunks, embeddings):
            record = {
                **chunk,  # Include all chunk data
                'embedding': embedding,
                'embedding_id': f"{chunk['original_row_id']}_{chunk['chunk_id']}"
            }
            medical_records.append(record)

        # Save to local JSONL first
        os.makedirs("embeddings", exist_ok=True)
        local_path = "embeddings/medical_embeddings.jsonl"

        with open(local_path, "w") as f:
            for record in medical_records:
                f.write(json.dumps(record, default=str) + "\n")

        print(f"✅ Saved {len(medical_records)} records to {local_path}")

        # Upload to GCS
        gcs_path = "embeddings/medical_embeddings.jsonl"
        blob = self.bucket.blob(gcs_path)
        blob.upload_from_filename(local_path)

        print(f"⬆️ Uploaded to gs://{self.bucket_name}/{gcs_path}")

        # Store in memory for immediate use
        self.medical_records = medical_records
        print(f"✅ Loaded {len(self.medical_records)} records into memory")

    def load_embeddings_from_gcs(self):
        """Load embeddings from GCS into memory"""
        try:
            gcs_path = "embeddings/medical_embeddings.jsonl"
            blob = self.bucket.blob(gcs_path)

            # Download to local file
            local_path = "/tmp/medical_embeddings.jsonl"
            blob.download_to_filename(local_path)

            # Load into memory
            self.medical_records = []
            with open(local_path, "r") as f:
                for line in f:
                    record = json.loads(line.strip())
                    self.medical_records.append(record)

            print(f"✅ Loaded {len(self.medical_records)} records from GCS")

        except Exception as e:
            print(f"❌ Error loading from GCS: {e}")
            print("Please run embedding creation first.")

    def search_similar_records(self, query: str, top_k: int = 10) -> List[Dict[str, Any]]:
        """Search for similar medical records using cosine similarity"""
        if not self.medical_records:
            print("❌ No medical records loaded. Please load embeddings first.")
            return []

        print(f"🔍 Searching for: '{query}'")

        # Generate query embedding
        query_embeddings = self.embedding_model.get_embeddings([query])
        query_vector = np.array([query_embeddings[0].values])

        # Get all record embeddings
        record_embeddings = np.array([record['embedding'] for record in self.medical_records])

        # Calculate similarities
        similarities = cosine_similarity(query_vector, record_embeddings)[0]

        # Get top-k similar records
        top_indices = np.argsort(similarities)[::-1][:top_k]

        similar_records = []
        for idx in top_indices:
            record = self.medical_records[idx].copy()
            record['similarity_score'] = float(similarities[idx])
            similar_records.append(record)

        return similar_records

    def extract_medical_codes_from_results(self, similar_records: List[Dict[str, Any]]) -> Dict[str, Dict]:
        """Extract and aggregate medical codes from search results"""
        medical_codes = {
            'diagnoses': {},
            'procedures': {},
            'cpt_codes': {}
        }

        for record in similar_records:
            # Process diagnoses
            for diag in record.get('diagnoses', []):
                code = diag.get('ICD9_CODE')
                if code:
                    if code not in medical_codes['diagnoses']:
                        medical_codes['diagnoses'][code] = {
                            'short_title': diag.get('SHORT_TITLE', ''),
                            'long_title': diag.get('LONG_TITLE', ''),
                            'frequency': 0,
                            'max_similarity': 0
                        }
                    medical_codes['diagnoses'][code]['frequency'] += 1
                    medical_codes['diagnoses'][code]['max_similarity'] = max(
                        medical_codes['diagnoses'][code]['max_similarity'],
                        record['similarity_score']
                    )

            # Process procedures
            for proc in record.get('procedures', []):
                code = str(proc.get('ICD9_CODE'))
                if code and code != 'None':
                    if code not in medical_codes['procedures']:
                        medical_codes['procedures'][code] = {
                            'short_title': proc.get('SHORT_TITLE', ''),
                            'long_title': proc.get('LONG_TITLE', ''),
                            'frequency': 0,
                            'max_similarity': 0
                        }
                    medical_codes['procedures'][code]['frequency'] += 1
                    medical_codes['procedures'][code]['max_similarity'] = max(
                        medical_codes['procedures'][code]['max_similarity'],
                        record['similarity_score']
                    )

            # Process CPT codes
            for cpt in record.get('cpt_codes', []):
                code = cpt.get('CPT_CD')
                if code:
                    if code not in medical_codes['cpt_codes']:
                        medical_codes['cpt_codes'][code] = {
                            'description': cpt.get('DESCRIPTION', ''),
                            'section': cpt.get('SECTIONHEADER', ''),
                            'subsection': cpt.get('SUBSECTIONHEADER', ''),
                            'frequency': 0,
                            'max_similarity': 0
                        }
                    medical_codes['cpt_codes'][code]['frequency'] += 1
                    medical_codes['cpt_codes'][code]['max_similarity'] = max(
                        medical_codes['cpt_codes'][code]['max_similarity'],
                        record['similarity_score']
                    )

        return medical_codes

    def generate_medical_response(self, query: str, similar_records: List[Dict[str, Any]], medical_codes: Dict[str, Dict]) -> str:
        """Generate explanation using Vertex AI Gemini"""

        # Prepare context from top records
        context = "\n".join([
            f"Record {i+1}: {record['content'][:300]}..."
            for i, record in enumerate(similar_records[:3])
        ])

        # Format medical codes
        diagnoses_text = ""
        if medical_codes['diagnoses']:
            diagnoses_text = "DIAGNOSIS CODES FOUND:\n"
            for code, info in sorted(medical_codes['diagnoses'].items(),
                                   key=lambda x: x[1]['max_similarity'], reverse=True)[:5]:
                diagnoses_text += f"- {code}: {info['long_title']} (frequency: {info['frequency']}, similarity: {info['max_similarity']:.3f})\n"

        procedures_text = ""
        if medical_codes['procedures']:
            procedures_text = "PROCEDURE CODES FOUND:\n"
            for code, info in sorted(medical_codes['procedures'].items(),
                                   key=lambda x: x[1]['max_similarity'], reverse=True)[:5]:
                procedures_text += f"- {code}: {info['long_title']} (frequency: {info['frequency']}, similarity: {info['max_similarity']:.3f})\n"

        cpt_text = ""
        if medical_codes['cpt_codes']:
            cpt_text = "CPT CODES FOUND:\n"
            for code, info in sorted(medical_codes['cpt_codes'].items(),
                                   key=lambda x: x[1]['max_similarity'], reverse=True)[:5]:
                cpt_text += f"- {code}: {info['description']} (frequency: {info['frequency']}, similarity: {info['max_similarity']:.3f})\n"

        prompt = f"""
You are a medical coding assistant. Based on the following medical records context and the user's query about diseases/symptoms, provide relevant medical codes and explanations.

CONTEXT FROM MEDICAL RECORDS:
{context}

{diagnoses_text}

{procedures_text}

{cpt_text}

USER QUERY: {query}

Please provide a comprehensive response that:
1. Explains what medical codes are most relevant to the query
2. Provides context on why these codes match the query
3. Lists the specific ICD9 diagnosis codes, procedure codes, and CPT codes
4. Explains what each code represents in simple terms
5. Notes the similarity scores to show relevance

Keep your response concise and well-formatted.
"""

        try:
            response = self.generative_model.generate_content(prompt)
            return response.text
        except Exception as e:
            return f"Error generating explanation: {e}. However, medical codes were found successfully."

    def query(self, user_query: str, top_k: int = 10) -> Dict[str, Any]:
        """Main query interface"""
        print(f"Processing query: {user_query}")

        # Search for similar records
        similar_records = self.search_similar_records(user_query, top_k)

        if not similar_records:
            return {
                'query': user_query,
                'explanation': "No similar medical records found.",
                'medical_codes': {'diagnoses': {}, 'procedures': {}, 'cpt_codes': {}},
                'retrieved_records_count': 0,
                'sample_records': []
            }

        # Extract medical codes
        medical_codes = self.extract_medical_codes_from_results(similar_records)

        # Generate explanation
        explanation = self.generate_medical_response(user_query, similar_records, medical_codes)

        # Prepare sample records
        sample_records = []
        for record in similar_records[:3]:
            sample_records.append({
                'content': record['content'][:200] + "..." if len(record['content']) > 200 else record['content'],
                'category': record['category'],
                'description': record['description'],
                'similarity_score': record['similarity_score'],
                'row_id': record['original_row_id']
            })

        return {
            'query': user_query,
            'explanation': explanation,
            'medical_codes': medical_codes,
            'retrieved_records_count': len(similar_records),
            'sample_records': sample_records
        }

In [20]:

def main():
    """Main execution function"""

    # Configuration
    CSV_PATH = "/content/ehr_records.csv"  # Update with your CSV path

    # Initialize components
    processor = MedicalDataProcessor()
    rag_system = VertexAIMedicalRAG(PROJECT_ID, LOCATION, BUCKET_NAME)

    # Setup mode
    setup_mode = input("Do you want to setup the database? (y/n): ").lower() == 'y'

    if setup_mode:
        print("=== DATA PROCESSING PHASE ===")
        df = processor.clean_csv_data(CSV_PATH)
        chunks = processor.chunk_documents(df)

        print("=== EMBEDDING & STORAGE PHASE ===")
        rag_system.create_embeddings_and_store(chunks)
        print("Setup complete!")
    else:
        # Load existing embeddings
        print("=== LOADING EMBEDDINGS FROM GCS ===")
        rag_system.load_embeddings_from_gcs()

    # Interactive query mode
    print("\n=== MEDICAL RAG SYSTEM READY ===")
    print("Enter medical queries to find relevant codes. Type 'quit' to exit.\n")

    # Sample queries
    sample_queries = [
        "chest pain and shortness of breath",
        "cardiac arrest and heart failure",
        "difficulty breathing",
        "hypotension and shock",
        "mechanical ventilation"
    ]

    print("Sample queries you can try:")
    for i, q in enumerate(sample_queries, 1):
        print(f"  {i}. {q}")
    print()

    while True:
        query = input("Enter your medical query (or 'quit' to exit): ").strip()
        if query.lower() in ['quit', 'exit', 'q']:
            break

        if not query:
            continue

        try:
            results = rag_system.query(query)

            print("\n" + "="*60)
            print(f"QUERY: {results['query']}")
            print("="*60)

            print("🏥 MEDICAL CODES FOUND:")

            # Display diagnoses
            if results['medical_codes']['diagnoses']:
                print("\n📋 DIAGNOSIS CODES (ICD9):")
                for code, info in results['medical_codes']['diagnoses'].items():
                    print(f"  • {code}: {info['long_title']} (freq: {info['frequency']}, sim: {info.get('max_similarity', 0):.3f})")

            # Display procedures
            if results['medical_codes']['procedures']:
                print("\n🔬 PROCEDURE CODES (ICD9):")
                for code, info in results['medical_codes']['procedures'].items():
                    print(f"  • {code}: {info['long_title']} (freq: {info['frequency']}, sim: {info.get('max_similarity', 0):.3f})")

            # Display CPT codes
            if results['medical_codes']['cpt_codes']:
                print("\n💊 CPT CODES:")
                for code, info in results['medical_codes']['cpt_codes'].items():
                    print(f"  • {code}: {info['description']} (freq: {info['frequency']}, sim: {info.get('max_similarity', 0):.3f})")

            if not any([
                results['medical_codes']['diagnoses'],
                results['medical_codes']['procedures'],
                results['medical_codes']['cpt_codes']
            ]):
                print("  No specific medical codes found for this query.")

            print(f"\n📄 EXPLANATION:")
            print(results['explanation'])

            print(f"\n📊 Retrieved {results['retrieved_records_count']} relevant medical records")
            print("="*60 + "\n")

        except Exception as e:
            print(f"❌ Error processing query: {e}")
            print("Please try again with a different query.\n")

if __name__ == "__main__":
    main()

✅ Initialized Vertex AI Medical RAG
   Project: i-monolith-468706-i9
   Location: us-central1
   Bucket: bucket_jonah
Do you want to setup the database? (y/n): y
=== DATA PROCESSING PHASE ===
Loading CSV data...
Original data shape: (1976, 14)
Cleaned data shape: (1945, 17)
Created 3710 text chunks
=== EMBEDDING & STORAGE PHASE ===
=== EMBEDDING CREATION PHASE ===
Generating embeddings for 3710 texts in batches of 32...
  Processed batch 1/116
  Processed batch 2/116
  Processed batch 3/116
  Processed batch 4/116
  Processed batch 5/116
  Processed batch 6/116
  Processed batch 7/116
  Processed batch 8/116
  Processed batch 9/116
  Processed batch 10/116
  Processed batch 11/116
  Processed batch 12/116
  Processed batch 13/116
  Processed batch 14/116
  Processed batch 15/116
  Processed batch 16/116
  Processed batch 17/116
  Processed batch 18/116
  Processed batch 19/116
  Processed batch 20/116
  Processed batch 21/116
  Processed batch 22/116
  Processed batch 23/116
  Processe